# LangChain 101: Ruby on Rails for Generative AI

[LangChain](https://langchain.com/) is a popular chatbot and LLM library that efficiently connects many tools. It is known as an easy way to produce chatbots, but is actually much more: an **onramp** to generative AI. LangChain is like [Ruby on Rails](https://en.wikipedia.org/wiki/Ruby_on_Rails) [which powered Web 2.0] for generative artificial intelligence. It has enabled the rapid adoption of LLM technology across the web, startup ecosystem and in the modern enterprise [big companies!].

#### Note: Credit to Ivan Reznikov

This content is indebted to [Ivan Reznikov](https://linkedin.com/in/reznikovivan), who created a [LangChain 101 Course](https://pub.towardsai.net/langchain-101-part-1-building-simple-q-a-app-90d9c4e815f3) that is excellent.

## Question & Answer (Q&A) using Retrieval Augmented Generation (RAG) with LangChain

The most popular techniques using large language models and LangChain are vector similarity search and Retrieval Augmented Generation (RAG) for Q&A models: indexing documents using a technique called _vector search_ and then retrieving parts of them relevant to a question, adding these document segments before the question's in a request, and submitting it to an LLM like [OpenAI](https://platform.openai.com/docs/introduction). **Talk is cheap**. We're going to start out by building a RAG engine using [ChromaDB](https://www.trychroma.com/) which is easy to get started with.

#### Note: Because the simple, local version of Chroma can be more difficult to scale, we will be using OpenSearch via Docker to work with larger sets of documents.

In [2]:
import logging
import os
from typing import Any, Dict, List, Optional, Type

import chromadb
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma

### Q&A [nerding out] about Network Motifs

I am obsessed with _network motifs_ - statistically significant patterns in _graphs_ (a graph in the real world is called a _network_) called _graphlets_ that appear in _complex networks_. You can see a simple and more complex heterogeneous, temporal network motif below.

<br />
<center>
    <img src="../images/5-graphlets.png" width="600px" />
</center>
<br />
<center>
    <a href="https://www.nature.com/articles/s41598-020-69795-1">Exploiting graphlet decomposition to explain the structure of complex networks: the GHuST framework</a>, Espejo et al., 2020
</center>
<br /><br />

<center><img src="../images/temporal-motifs.png" width="400px" /></center>
<br />
<center><a href="https://snap.stanford.edu/temporal-motifs/">Motifs in temporal networks</a>, Ashwin Paranjape, Austin R. Benson, and Jure Leskovec., 2017</center>
<br />

If you run the [`./download.sh`](download.sh) script in a terminal window, it will download a tarball of 25 network motif datasets and extract them into the `data/Network_Motifs/` folder. We will use ChromaDB to implement RAG over these systems before getting into a deeper explanation of what is going on and what is available in Langchain.

#### Load a copy of one of my Dropbox folders with academic papers

In [5]:
PAPER_FOLDER = f"{os.getcwd()}/../data/Network_Motifs/"

#### Verify papers directory

In [6]:
paper_count = len(os.listdir(PAPER_FOLDER))
print(f"You have {paper_count:,} Network Motif PDFs in `{PAPER_FOLDER}`.")

You have 25 Network Motif PDFs in `/home/jovyan/work/Part 1. Langchain/../data/Network_Motifs/`.


#### Load our OpenAI key

In [7]:
# Set in env/openai.env
openai_api_key = os.environ.get("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

#### Load all PDFs from academic paper folder

In [8]:
loader = PyPDFDirectoryLoader(PAPER_FOLDER, silent_errors=True)
docs = loader.load()
print(f"You have {len(docs)} document segments in `{PAPER_FOLDER}`.")

You have 661 document segments in `/home/jovyan/work/Part 1. Langchain/../data/Network_Motifs/`.


#### How many papers on network motifs?

In [9]:
motif_docs = [doc for doc in docs if "motif" in doc.page_content]
motif_doc_count = len(motif_docs)
paper_count = len(set(doc.metadata["source"] for doc in motif_docs))
print(
    f"You have {paper_count} papers mentioning network motifs split across {motif_doc_count} document segments in `{PAPER_FOLDER}`."
)

You have 20 papers mentioning network motifs split across 321 document segments in `/home/jovyan/work/Part 1. Langchain/../data/Network_Motifs/`.


#### Embed them with OpenAI ada model and store them in OpenSearch

In [10]:
embeddings = OpenAIEmbeddings()
fs = LocalFileStore("./data/embedding_cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings, fs, namespace=embeddings.model,
)

#### Load it into Chroma from our documents set

In [37]:
chromadb = Chroma.from_documents(docs, cached_embedder)

### Similarity Search

If our task is information retrieval - semantic search - returning documents related to the meaning of a text query, LangChain and ChromaDB have us covered.

#### Let's retrieve documents similar to a question, a part of Q&A

In [38]:
query = "What is a network motif?"
docs = chromadb.similarity_search(query)
docs[0]

Document(page_content='8How do we findmodules of network motifs?', metadata={'page': 7, 'source': '/home/jovyan/work/Part 1. Langchain/../data/Network_Motifs/Higher-Order Organization of Complex Networks - Slides from 2016.pdf'})

#### Let's look for documents that talk about heterogeneous network motif computation complexity

In [40]:
query = "Hetereogeneous network motif compute complexity"
docs = chromadb.similarity_search(query)
docs[0]

Document(page_content='Heterogeneous Network Motifs\n[5]Nesreen K. Ahmed, Ryan A. Rossi, Theodore L. Willke, and Rong Zhou. 2017.\nA Higher-order Latent Space Network Model. In Proceedings of the AAAI PAIR\n(Plan, Activity, and Intent Recognition) Workshop . 1–7.\n[6]Nesreen K. Ahmed, Ryan A. Rossi, Rong Zhou, John Boaz Lee, Xiangnan Kong,\nTheodore L. Willke, and Hoda Eldardiry. 2018. Learning Role-based Graph\nEmbeddings. In StarAI IJCAI .\n[7]Nesreen K. Ahmed, Theodore L. Willke, and Ryan A. Rossi. 2016. Estimation of\nLocal Subgraph Counts. In Proceedings of the IEEE International Conference on\nBigData . 586–595.\n[8]Leman Akoglu, Hanghang Tong, and Danai Koutra. 2015. Graph based anomaly\ndetection and description: a survey. DMKD 29, 3 (2015), 626–688.\n[9]Danielle Smith Bassett and ED Bullmore. 2006. Small-world brain networks. The\nneuroscientist 12, 6 (2006), 512–523.\n[10] Austin R Benson, David F Gleich, and Jure Leskovec. 2016. Higher-order organi-\nzation of complex networ

### LangChain Q&A with ChromaDB

Now let's do Q&A about network motifs using OpenAI. You can find the different OpenAI models you can use in the [Models docs](https://platform.openai.com/docs/models).

#### Setup a simple buffer memory system to submit with the API calls to provide prompt context

In [58]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#### Create a ConversationalRetrievalChain from the LLM, the vectorstore, and the memory system

In [113]:
qa = ConversationalRetrievalChain.from_llm(
    # OpenAI(model="text-davinci-003", temperature=0.8),
    ChatOpenAI(temperature=0.8, model="gpt-3.5-turbo-16k"),
    chromadb.as_retriever(),
    memory=memory,
    verbose=True,
)

#### Lets ask it about network motifs!

In [114]:
# print(qa({"question": "What is a simple network motif?"})["answer"])

#### Fantastic! I get an excellent definition :)

In [115]:
# print(qa({"question": "What is a graphlet? What are orbits?"})["answer"])

In [116]:
# print(qa({"question": "What are the different types of network motif?"})["answer"])

In [117]:
retriever = chromadb.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [118]:
retrieved_docs = retriever.get_relevant_documents(
    "How do you search for network motifs in a network?"
)

In [119]:
retrieved_docs[0:5]

[Document(page_content='8How do we findmodules of network motifs?', metadata={'page': 7, 'source': '/home/jovyan/work/Part 1. Langchain/../data/Network_Motifs/Higher-Order Organization of Complex Networks - Slides from 2016.pdf'}),
 Document(page_content='networks [14], financial [15] and trade networks [16], and internet and mobile wireless com-\nmunication [17]. This hasledtoawhole range (quite likely hundreds) ofsoftware toolboxes or\none-off programs fordetecting network motifs [18]. This paper outlines briefly theorigins and\nhistory ofnetwork motifs and themain algorithm foridentifying motifs, that is,before its\nrecent “rediscovery” [19] (Merton 1961) attheturn ofthemillennia byMilo and colleagues\n(2002) [1]and Shen-Orr and colleagues (2002) [3].\nInorder toproceed, wedefine afewbasic terms from network theory. Any network or\ngraph may bestudied interms ofitsbinary adjacency matrixA.Foranetwork withnnodes,\nand ann×nbinary adjacency matrixA,thenAij=1implies that node-i isconne

# What is LangChain?

#### Note: the following LangChain introduction is originally by [Ivan Reznikov](https://linkedin.com/in/rez) in [LangChain 101: Part 1. Building Simple Q&A App](https://pub.towardsai.net/langchain-101-part-1-building-simple-q-a-app-90d9c4e815f3).

Today, we will discuss the following topics:

* What exactly is LangChain?
* LangChain’s fundamental concepts and components
* How to build a basic LangChain application

Lang stands for language, which is the primary focus of LangChain, and chain — the connotation of connecting things — refers to the chain component used in LangChain. Chains are sequences of instructions that the framework executes to perform a task. This simplifies the use of Large Language Models for specific tasks and enables you to combine the power of LLMs (Large Language Models) with other programming techniques.

I’ve been asked how LangChain differs from ChatGPT or LLM. To answer this question, I’m attaching a table that highlights the differences:

<pre><code>
+==========+========================+====================+====================+
|          | LangChain              | LLM                | ChatGPT            | 
+==========+========================+====================+====================+
| Type     | Framework              | Model              | Model              | 
+----------+------------------------+--------------------+--------------------+
| Purpose  | Build applications     | Generate text      | Generate chat      | 
|          | with LLMs              |                    | conversations      | 
+----------+------------------------+--------------------+--------------------+
| Features | Chains, prompts, LLMs, | Large dataset of   | Large dataset of   | 
|          | memory, index, agents  | text and code      | chat conversations | 
+----------+------------------------+--------------------+--------------------+
| Pros     | Can combine LLMs with  | Generates nearly   | Generates realistic| 
|          | programming techniques | human-quality text | chat conversations | 
+----------+------------------------+--------------------+--------------------+
| Cons     | Requires some          | Not as easy to use | Not as versatile   | 
|          | programming knowledge  | for specific tasks | as LangChain       | 
+----------+------------------------+--------------------+--------------------+
</code></pre>

## Basic components of LangChain

There are six basic components of Langchain:

- Models
- Prompts
- Chains
- Memory
- Indexes
- Agents and Tools

Let’s briefly talk about these components.

<br />
<center><img src="../images/6-langchain-components.webp" width="600px" /></center>

### Models

Models in LangChain are large language models (LLMs) trained on enormous amounts of massive datasets of text and code.

Models are used in LangChain to generate text, answer questions, translate languages, and much more. They are also used to store information that the framework can access later.

Examples: GPT-x, Bloom, Flan T5, Alpaca, LLama, Dolly, FastChat-T5, etc.

<br />
<center><img src="../images/langchain-models.webp" width="600px" /></center>

### Prompts

Prompts are pieces of text that guide the LLM to generate the desired output. Prompts can be simple or complex and can be used for text generation, translating languages, answering questions, and more.

In LangChain, prompts play a vital role in controlling the output of the LLM. You can influence the LLM to generate the desired output by carefully crafting the prompt.

Here are some examples of how prompts can be used:

* Specify the desired output format: You can, for example, use a prompt to instruct the LLM to generate text, translate languages, or answer questions. Example: Translate the input to Arabic
* Provide context: A prompt can provide context for the LLM, such as information about the output topic or examples of the desired output. Example: Explain the answer step-by-step like a school teacher
* Constrain the output: You can use a prompt to limit the LLM’s output by specifying a maximum length or selecting a list of keywords to include in the output. Example: Generate a tweet post using less than 140 words

<br />
<center><img src="../images/langchain-prompts.webp" width="600px" /></center>

### Indexes

Indexes are unique data structures to store information about the data content. This information can include terms found in each document, document location in the dataset, relationships between documents, etc. Vectorstores are data structures storing vector representations of a dataset’s terms. A retriever is an interface that returns documents in response to an unstructured query. It is broader in scope than a vector store.

<center><img src="../images/langchain-indexes.webp" width="600px" /></center>
<br />

Understanding indexes, vectorstores, and retrievers is key to building an app on your specific data. Consider another example of a chain that includes all three components:

A chain is formed to answer financial questions.

* The chain uses an index to find all documents that contain the word “finance.”
* The chain uses a vectorstore to find other terms that are most similar to the word “finance” (“money,” “investments,” etc.).
* The chain uses a retriever to retrieve the documents that are ranked highest for the query “What are ways to invest?”.

### Memory

Memory in LangChain is a method of storing data that the LLM can later access. This information can include previous chain results, the context of the current chain, and any other information that the LLM requires. It also enables the application to keep track of the current conversation’s context.

<center><img src="../images/langchain-memory.webp" width="600px" /></center>
<br />

Memory plays a vital role in LangChain, allowing the LLM to learn from previous interactions and build up a knowledge base. This knowledge base can then be used to improve the performance of future chains.

Consider the creation of a chain designed to answer real estate-related questions. The chain may use memory to store the results of previous chains that have responded to real estate questions. When the chain asked a new question about a similar topic, this information could be used to improve its performance.

### Chains

Chains are sequences of instructions the LangChain framework executes to perform a task. Chains may connect other LangChain components based on the application requirements. They allow the framework to perform a wide variety of tasks.

<center><img src="../images/langchain-chains.webp" width="600px" /></center>
<br />

Assume we’re creating an app to assist us in interview preparation:

* Prompt: “I’m getting ready for an interview for a position as a software engineer. Can you ask me some common interview questions that I may expect?”
* Function A: This function would access the LLM’s knowledge of the software engineering field, such as its knowledge of common interview questions for software engineers. It can also look for appropriate data in the vectorstore.
* Function B: This function would manipulate data, such as generating a list of common interview questions for software engineers or a list of resources to help the student prepare for the interview. It will select a question and ask it.
* Memory: some follow-up questions might be asked to help better understand the knowledge of the chosen topic. Memory implementation will allow the chain to keep the context of the conversation.
 
### Agents and Tools

Agents and tools are two important concepts in LangChain.

<center><img src="../images/langchain-agents-tools.webp" width="600px" /></center>
<br />

Agents are reusable components that can perform specific tasks such as text generation, language translation, and question-answering. Tools are function libraries that can be used to aid in developing various agents.

Examples:

* NewsGenerator agent — for generating news articles or headlines.
* DataManipulator tool — for manipulating data (cleaning, transforming, or extracting features).
* Application. Building a Q&A system.
* Colab and Github links.

Let’s build a Q&A system to answer our questions regarding official holidays in the United Arab Emirates. Based on the components we’ve learned, we can draw a schema similar to the following one below:

First, we install all dependencies. Some of the versions might be updated, but installing a specific version will resolve most, if not all, inner dependencies.

### Application. Explaining LangChain by Building a Q&A system.

Let’s build a Q&A system to answer our questions regarding official holidays in the United Arab Emirates. Based on the components we’ve learned, we can draw a schema similar to the following one below:

<center><img src="../images/langchain-q-and-a.webp" width="600px" /></center>
<br />

Note that all dependencies are provided on the Docker image via [`Dockerfile`](Dockerfile) and [`docker-compose.yml`](docker-compose.yml). Some of the versions might be updated, but installing a specific version will resolve most, if not all, inner dependencies.

## OpenAI Text Models

You can see the OpenAI models available through their APIs at [https://platform.openai.com/docs/models/](https://platform.openai.com/docs/models/).

In [120]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# llm = OpenAI(temperature=0.8, model="text-davinci-003")
llm = ChatOpenAI(temperature=0.8, model="gpt-3.5-turbo-16k")

template = """Question: {question}

Answer: Let's give a concise answer using our science skills."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is a simple network motif?"

llm_chain.run(question)

'A simple network motif refers to a recurring pattern or subgraph that occurs frequently in a network. These motifs typically involve a small number of nodes and edges and can have specific functions or structural properties within the network. Examples of simple network motifs include feed-forward loops, feedback loops, and bipartite motifs.'

In [121]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [122]:
llm = ChatOpenAI(temperature=0.8, model="gpt-3.5-turbo-16k")

In [123]:
template = """Question: {question}

Answer: Let's give a concise answer using our science skills."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [124]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [125]:
question = "What is a simple network motif?"

llm_chain.run(question)

'A simple network motif refers to a recurring pattern or structure found in a network or graph. It typically involves a small set of connected nodes and their interactions. These motifs can provide insights into the underlying principles and dynamics of complex networks, such as biological systems or social networks. Simple network motifs often represent fundamental building blocks that contribute to the overall behavior and function of the larger network.'